In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [104]:
pip install openai


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [105]:
pip install mistralai


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
api_key = os.environ["MISTRAL_API_KEY"]
openrouter_api_key = os.environ["OPENROUTER_API_KEY"]

In [3]:
from openai import OpenAI
import json

In [4]:
learnLM_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openrouter_api_key,
)

In [5]:
from mistralai import Mistral

client = Mistral(api_key=api_key)
model = "mistral-large-latest"


In [110]:
def check_answer_prompt(question, expected_answer, user_answer):
    prompt = [{"role":"system", "content":f"""
    You are an AI that checks answers for correctness.

    - The question is: "{question}"
    - The expected answer is: "{expected_answer}"
    - The user's answer is: "{user_answer}"

    Based on the user's answer compared to the expected answer, determine if the user's answer is **correct** or **incorrect**.

    Output only in valid JSON format like this:

    {{
        "result": "correct"  # or "incorrect"
    }}
    """}]

    completion = learnLM_client.chat.completions.create(

      model="google/learnlm-1.5-pro-experimental:free",
      messages=prompt
    )

    response = completion.choices[0].message.content
    start_index = response.find('{')
    end_index = response.rfind('}') + 1
    json_text = response[start_index:end_index]

    return json.loads(json_text)

In [111]:
def followup_que_gen(student_question):
    messages = [{"role": "system",
                 "content": f"""You are a Socratic tutor. This is the student question: '{student_question}'.
                 Please generate **an appropriate number of follow-up questions**, each with an expected answer, to guide the student to the correct answer.
                 Only generate the number of questions and answers that are necessary, up to a maximum of 10.
                 If fewer questions are sufficient, stop at the appropriate point.

                 Output the follow-up questions and their expected answers in **valid JSON format** like this:

                 {{
                     "questions_and_answers": [
                         {{"question": "What is the first step in solving the equation?",
                           "answer": "Isolate the variable by subtracting 3 from both sides."}},
                         {{"question": "Can you identify the terms?",
                           "answer": "The terms are 2x and 3 on the left side of the equation."}},
                         ...
                     ]
                 }}

                 Ensure the output is **valid JSON**, not truncated, and contains an appropriate number of questions and their corresponding expected answers, based on the complexity of the student's question."""
                }]

    completion = learnLM_client.chat.completions.create(

      model="google/learnlm-1.5-pro-experimental:free",
      messages=messages
    )

    response = completion.choices[0].message.content

    start_index = response.find('{')
    end_index = response.rfind('}') + 1
    json_text = response[start_index:end_index]

    return json.loads(json_text)

In [114]:
# current_question_index = 0 
# questions_and_answers = []

# def tutoring(student_answer, question, expected_answer,):
    
#     if current_question_index < len(questions_and_answers):
#       int_response = check_answer_prompt(question, expected_answer, student_answer)
#       is_correct = int_response.get("result") == "correct"

#       messages.append({"role": "user", "content":student_answer})
#       if is_correct:
#           print(questions_and_answers[current_question_index])
#           messages.append({"role": "tutor", "content": questions_and_answers[current_question_index]})
#           current_question_index +=1
#       else:
#           print("Try again!")
#           messages.append({"role": "tutor", "content": "Try again!"})


#       completion = learnLM_client.chat.completions.create(

#         model="google/learnlm-1.5-pro-experimental:free",
#         messages=messages
#       )

#       return completion.choices[0].message.content
    
#     else:
#         print("over...")

In [115]:
student_question = "It has been estimated that it will take 10 men 6 days to complete a certain task. Find the number of days it will take 8 men to complete a job which is double that task. Student Question: It has been estimated that it will take 10 men 6 days to complete a certain task. Find the number of days it will take 8 men to complete a job which is double that task."

In [116]:
response = followup_que_gen(student_question)
print(response)

{'questions_and_answers': [{'question': 'If it takes 10 men 6 days to complete a task, how much work is done by one man in one day?', 'answer': 'Each man does 1/(10*6) = 1/60 of the task in one day.'}, {'question': 'Knowing that one man does 1/60 of the task per day, how much work can 8 men do in one day?', 'answer': '8 men can do 8 * (1/60) = 8/60 = 2/15 of the task in one day.'}, {'question': 'The problem states the new job is double the original task.  If the original task is represented by 1, how much work is the new job?', 'answer': 'The new job is double the original, so it represents 2 times the work, or 2.'}, {'question': 'If 8 men can complete 2/15 of the task in one day, and the new job is twice the size of the original task, how many days will it take them to complete the doubled task?', 'answer': 'It will take them 2 / (2/15) = 2 * (15/2) = 15 days.'}]}


In [7]:
socratic_tutor_description = "Act as a socratic totor"

In [8]:
socratic_tutor_properties = \
{
    # arg 1: section of memory to edit
    "student_question": {
        "type": "string",
        "description": "Student question which is need to be act as a socratic tutor",            
    }
}

In [119]:
# core_tutoring_description = "generate suitable response for the student answer"

In [120]:
# core_tutoring_properties = \
# {
#     "student_answer":{
#         "type":"string",
#         "description":"Student answer which is need to give response"
#     }
# }

In [9]:
# tool schema (passed to OpenAI)
tool_metadata = [
    {
        "type": "function",
        "function": {
            "name": "socratic_tutor",
            "description": socratic_tutor_description,
            "parameters": {
                "type": "object",
                "properties": socratic_tutor_properties,
                "required": ["student_question"],
            },
        }
    },
    # {
    #     "type": "function",
    #     "function": {
    #         "name": "tutoring",
    #         "description": core_tutoring_description,
    #         "parameters": {
    #             "type": "object",
    #             "properties": core_tutoring_properties,
    #             "required": ["student_answer"],
    #         },
    #     }
    # }
]

In [10]:
system_prompt = "You are a Educational Tutor called LAAI. You must either call  " \
    "tools (socratic_tutor) or write response to the user. Do not take the same actions " \
    "multiple times! When student ask maths related question, make sure to always call the " \
    "socratic_tutor tool."

In [11]:
import json

In [155]:
messages = [
        # system prompt 
        {"role": "system", "content": system_prompt}, 
] 

current_question_index = 0 
questions_and_answers = []

def tutoring(student_answer, question, expected_answer,):
    global current_question_index

    if current_question_index < len(questions_and_answers):
      int_response = check_answer_prompt(question, expected_answer, student_answer)
      is_correct = int_response.get("result") == "correct"

      messages.append({"role": "user", "content":student_answer})
      if is_correct:
          print(questions_and_answers[current_question_index])
          messages.append({"role": "assistant", "content": questions_and_answers[current_question_index]})
          current_question_index +=1
      else:
          print("Try again!")
          messages.append({"role": "assistant", "content": "Try again!"})


      completion = learnLM_client.chat.completions.create(

        model="google/learnlm-1.5-pro-experimental:free",
        messages=messages
      )

      return completion.choices[0].message.content
    
    else:
        print("over...")
        return("over...")
    

# current_question_index = 0 



In [159]:
def agent_step(user_message):
    # user_message = input()
    global current_question_index

    messages.append({"role": "user", "content": user_message})


    while True:
        chat_completion = client.chat.complete(
            model=model,
            messages=messages,
            tools=tool_metadata
        )
        response = chat_completion.choices[0]

        # update the messages with the agent's response 
        messages.append(response.message)


        # if NOT calling a tool (responding to the user), return 
        if not response.message.tool_calls: 
            return response.message.content
        
        else: 
            print("TOOL CALL:", response.message.tool_calls[0].function)
            
            # parse the arguments from the LLM function call
            arguments = json.loads(
                response.message.tool_calls[0].function.arguments
            )

            if response.message.tool_calls[0].function.name == "followup_que_gen":
                
                # run the function with the specified arguments
                followup_que = followup_que_gen(**arguments)
                print(followup_que)
                
                questions_and_answers = followup_que.get("questions_and_answers", [])
                if not questions_and_answers:
                    raise ValueError("No follow-up questions generated")
                
                # for item in questions_and_answers:
                question = questions_and_answers[current_question_index].get("question")
                expected_answer = questions_and_answers[current_question_index].get("answer")

                if not question or not expected_answer:
                    continue

                print(question)
                   
                current_question_index += 1
                messages.append({"role": "assistant", "content": question})
            
            if response.message.tool_calls[0].function.name == "tutoring":
                tutoring_res = tutoring(**arguments)
                print(tutoring_res)

                



            # add the tool call response to the message history 
            messages.append({
                "role": "tool", 
                "tool_call_id": response.message.tool_calls[0].id, 
                "name": "followup_que_gen", 
                "content": f"Updated memory: {response}"
            })


In [125]:
agent_step("Hi")

"Hello! I'm LAAI, your educational tutor. I'm here to help you with any questions or topics you're studying. How can I assist you today?"

In [126]:
agent_step("Who are you?")

"I'm LAAI, your educational tutor. I'm here to help you with any questions or topics you're studying. How can I assist you today?"

In [127]:
agent_step("what color is apple")

'Apples can be various colors, including red, green, and yellow, depending on the variety.'

In [160]:
agent_step("can you help me to slove this question: It has been estimated that it will take 10 men 6 days to complete a certain task. Find the number of days it will take 8 men to complete a job which is double that task. Student Question: It has been estimated that it will take 10 men 6 days to complete a certain task. Find the number of days it will take 8 men to complete a job which is double that task.")

SDKError: API error occurred: Status 400
{"object":"error","message":"Unexpected role 'user' after role 'tool'","type":"invalid_request_error","param":null,"code":null}